<a href="https://colab.research.google.com/github/Architg021/SUNRISE/blob/main/Sentinment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install numpy==1.25.2 --force-reinstall
#!pip install --upgrade matplotlib
#!pip install --upgrade pandas
#!pip install --upgrade scikit-learn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.decomposition import IncrementalPCA
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
#!pip install tensorflow
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
!pip show tensorflow

Name: tensorflow
Version: 2.18.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tf_keras


1. Load Dataset

In [2]:
df = pd.read_csv("/content/Twitter_Data.csv")

In [3]:
# Check for null values and drop them
df.dropna(subset=['clean_text', 'category'], inplace=True)

 2. Text Preprocessing

In [4]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

nltk.download('stopwords')
nltk.download('punkt')

# Pre-compile regular expression
pattern = re.compile(r'[^a-zA-Z\s]')

# Create a set of stopwords for faster lookup
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation and special characters
    text = pattern.sub('', text)  # Use pre-compiled pattern

    # Tokenize the text and remove stopwords in one step
    tokens = [word for word in word_tokenize(text) if word not in stop_words]

    return ' '.join(tokens)

df['cleaned_text'] = df['clean_text'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [6]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Download necessary NLTK data packages
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab') # Download the missing punkt_tab data package

# Pre-compile regular expression
pattern = re.compile(r'[^a-zA-Z\s]')

# Create a set of stopwords for faster lookup
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation and special characters
    text = pattern.sub('', text)  # Use pre-compiled pattern

    # Tokenize the text and remove stopwords in one step
    tokens = [word for word in word_tokenize(text) if word not in stop_words]

    return ' '.join(tokens)

df['cleaned_text'] = df['clean_text'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


3. Feature Engineering

In [7]:
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_tfidf = vectorizer.fit_transform(df['cleaned_text']).toarray()
y = df['category']

4. Split Dataset

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [ ]:
# Apply Incremental PCA for dimensionality reduction
n_batches = 10  # Adjust based on your data size and memory capacity
# Set n_components to an integer or None
inc_pca = IncrementalPCA(n_components=None, # or a specific number of components, e.g., 100
                         batch_size=X_train.shape[0] // n_batches)

X_train_pca = inc_pca.fit_transform(X_train)
X_test_pca = inc_pca.transform(X_test)


5. Machine Learning Models

Naive Bayes

In [ ]:
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
y_pred_nb = nb_model.predict(X_test)

Support Vector Machine

In [ ]:
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)